In [5]:
import http.client
import json
import time
import csv
import datetime
from datetime import timedelta
import re
from tracemalloc import start
import pandas as pd
import os
import zipfile
import certifi
import ssl
import sys

In [ ]:

class NRGStreamApiOutageDetect:

    def __init__(self,username=None,password=None):
            self.username = '*******'
            self.password = '****************'                
            self.server = 'api.nrgstream.com'        
            self.tokenPath = '/api/security/token'
            self.releasePath = '/api/ReleaseToken'
            self.tokenPayload = f'grant_type=password&username={self.username}&password={self.password}'
            self.tokenExpiry = datetime.now() - timedelta(seconds=60)
            self.accessToken = ""
                 
    def getToken(self):
            try:
                if self.isTokenValid() == False:                             
                    headers = {"Content-type": "application/x-www-form-urlencoded"}      
                    # Connect to API server to get a token
                    context = ssl.create_default_context(cafile=certifi.where())
                    conn = http.client.HTTPSConnection(self.server,context=context)
                    conn.request('POST', self.tokenPath, self.tokenPayload, headers)
                    res = conn.getresponse()                
                    res_code = res.code
                    # Check if the response is good
                    
                    if res_code == 200:
                        res_data = res.read()
                        # Decode the token into an object
                        jsonData = json.loads(res_data.decode('utf-8'))
                        self.accessToken = jsonData['access_token']                         
                        # Calculate new expiry date
                        self.tokenExpiry = datetime.now() + timedelta(seconds=jsonData['expires_in'])                        
                        #print('token obtained')
                        #print(self.accessToken)
                    else:
                        res_data = res.read()
                        print(res_data.decode('utf-8'))
                    conn.close()                          
            except Exception as e:
                print("getToken: " + str(e))
                # Release token if an error occured
                self.releaseToken()  

    def releaseToken(self):
            try:            
                headers = {}
                headers['Authorization'] = f'Bearer {self.accessToken}'            
                context = ssl.create_default_context(cafile=certifi.where())
                conn = http.client.HTTPSConnection(self.server,context=context)
                conn.request('DELETE', self.releasePath, None, headers)  
                res = conn.getresponse()
                res_code = res.code
                if res_code == 200:   
                    # Set expiration date back to guarantee isTokenValid() returns false                
                    self.tokenExpiry = datetime.now() - timedelta(seconds=60)
                    #print('token released')            
            except Exception as e:
                print("releaseToken: " + str(e))
                        
    def isTokenValid(self):
            if self.tokenExpiry==None:
                return False
            elif datetime.now() >= self.tokenExpiry:            
                return False
            else:
                return True            

    def GetStreamDataByStreamId(self,streamIds, fromDate, toDate, dataFormat='csv', dataOption=''):
            stream_data = "" 
            # Set file format to csv or json            
            DataFormats = {}
            DataFormats['csv'] = 'text/csv'
            DataFormats['json'] = 'Application/json'
            try:                            
                for streamId in streamIds:            
                    # Get an access token            
                    self.getToken()    
                    if self.isTokenValid():
                        # Setup the path for data request. Pass dates in via function call
                        path = f'/api/StreamData/{streamId}'
                        if fromDate != '' and toDate != '':
                            path += f'?fromDate={fromDate.replace(" ", "%20")}&toDate={toDate.replace(" ", "%20")}'
                        if dataOption != '':
                            if fromDate != '' and toDate != '':
                                path += f'&dataOption={dataOption}'        
                            else:
                                path += f'?dataOption={dataOption}'        
                        
                        # Create request header
                        headers = {}            
                        headers['Accept'] = DataFormats[dataFormat]
                        headers['Authorization']= f'Bearer {self.accessToken}'
                        
                        # Connect to API server
                        context = ssl.create_default_context(cafile=certifi.where())
                        conn = http.client.HTTPSConnection(self.server,context=context)
                        conn.request('GET', path, None, headers)
                        res = conn.getresponse()        
                        res_code = res.code                    
                        if res_code == 200:   
                            try:
                                print(f'{datetime.now()} Outputing stream {path} res code {res_code}')
                                # output return data to a text file            
                                if dataFormat == 'csv':
                                    stream_data += res.read().decode('utf-8').replace('\r\n','\n') 
                                elif dataFormat == 'json':
                                    stream_data += json.dumps(json.loads(res.read().decode('utf-8')), indent=2, sort_keys=False)
                                conn.close()

                            except Exception as e:
                                print(str(e))            
                                self.releaseToken()
                                return None  
                        else:
                            print(str(res_code) + " - " + str(res.reason) + " - " + str(res.read().decode('utf-8')))
                        
                    self.releaseToken()   
                    # Wait 1 second before next request
                    time.sleep(1)
                return stream_data        
            except Exception as e:
                print(str(e))    
                self.releaseToken()
                return None

    def csvStreamToPandas(self, streamData):
        # split lines of return string from api
        streamData = streamData.split("\n")
        
        # remove empty elements from list
        streamData = [x for x in streamData if len(x) > 0] 
        
        # remove header data
        streamData = [x for x in streamData if x[0] != '#'] 
                     
        # split elements into lists of lists                     
        streamData = [x.split(",") for x in streamData] 
        
        # create dataframe
        df = pd.DataFrame(streamData[1:], columns=streamData[0]) 
        
        return df

try:    
    
    
    #credentials = f.readline().split(',')
    reedus = settings.TwentyFourMonthAPI
    #reedus = int(reedus)
    #f.close()
    nrgStreamApi = NRGStreamApiOutageDetect(settings.USERNAME,settings.PASSWORD)         
    # Date range for your data request
    # Date format must be 'mm/dd/yyyy hh:ss'
    fromDateStr = settings.DATE_TODAYSLASH
    toDateStr = settings.DATE_TWENTYFOURSLASH

    # Specify output format - 'csv' or 'json'
    dataFormat = 'csv'
    
    # Convert streams to Pandas dataframes
    # Only compatible with getByStream and getByFolder
    dataFrameConvert = True
    
    # Data Option
    dataOption = ''
    
    # Output from the API request is written to the stream_data variable
    stream_data = ""

    # Output from the API request is written to the streamList variable
    streamList = ""
    
    # Output from the API request is written to the folderList variable
    folderList = ""
    
    # Output from the API request is written to the groupExtractsList variable
    groupExtractsList = ""

    # Change to True to get streams by Stream Id
    getByStream = True    
    if getByStream:
        # Pass in individual stream id
        streamIds = [reedus]       
        # Or pass in list of stream ids
        #streamIds = [139308, 3, 225, 4117, 17, 545, 40034]         
        stream_data = nrgStreamApi.GetStreamDataByStreamId(streamIds, fromDateStr, toDateStr, dataFormat, dataOption) 
        
        if(dataFrameConvert and dataFormat == 'csv'):
            if(len(streamIds) > 1):
                print('Please only convert 1 stream to a Pandas dataframe at a time')
            else:
                stream_data = nrgStreamApi.csvStreamToPandas(stream_data)
        print (stream_data)
        
    # Change to True to get streams by Folder Id
    getByFolder = False
    if getByFolder:
        # Pass in individual folder id
        folderId = 9
        nrgStreamApi.GetStreamDataByFolderId(folderId, fromDateStr, toDateStr, dataFormat)

    # Change to True to retrieve a list of data options available for a given stream
    getStreamDataOptions = False
    if getStreamDataOptions:          
        # Pass in a list of streamIds to get the shapes available for each
        # These shapes can be passed to StreamData endpoint as 'displayOption' to retrieve only that shape
        streamIds = [2270]    
        streamDataOptions = nrgStreamApi.StreamDataOptions(streamIds, dataFormat)
        print(streamDataOptions)
  
     
except Exception as e:
    print(str(e))

In [ ]:
       
def return_graph(final_difference):
    final_difference = final_difference
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=final_difference['Expected Supply'],x = final_difference['Effective Date'],hoverinfo='x+y',
                            mode = 'lines',
                            fill='tozeroy'))

    fig.update_layout(
        
        title="24 Month Supply and Demand - Daily Peak Hour",
        xaxis_title="Date",
        yaxis_title="MW Surplus",
        legend_title="",
        font=dict(
            family="Courier New, monospace",
            size=15,
            color="green"
        ))

    fig.update_layout(
        title={
            'text': "24 Month Supply and Demand - Daily Peak Hour",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    fig.write_html(file = 'ss.html')
    hti = Html2Image()
    with open('./ss.html') as f:
        hti.screenshot(f.read(), save_as='offlineSupply.png')
    return fig.show()
def saves_Today(stream_data):
    stream_data = stream_data
    csvline= str(datetime.datetime.now().date() + datetime.timedelta(days=0))
    stream_data.set_index('Effective Date',inplace=True)
    stream_data = stream_data.apply(pd.to_numeric)
    stream_data.to_pickle(csvline +"dummyOneDayBehind.pkl")
    filenames = csvline + "dummyOneDayBehind.pkl"
    source = 'C:\\Users\\**********\\OneDrive - ****************\\Desktop\\PythonRepo' + '\\' + str(filenames)
    destination = "C:\\Users\\**********\\OneDrive - ****************\\Desktop\\PythonRepo\\AnnualDetectionChange"
    dest = shutil.copy(source, destination)
    print(csvline)
    return stream_data

def gets_Yesterday():
    ##this CSVline Delta does not change what dates you are querying. It changes what the file is saved as.
    ## to change the file data query data, you must change settings.py
    csvlineDelta = datetime.datetime.now().date()
    csvlineDelta=csvlineDelta + datetime.timedelta(days=-1)
    csvlineDelta = str(csvlineDelta)
    df_yesterday = pd.read_pickle(csvlineDelta+"dummyOneDayBehind.pkl")
    df_yesterday.drop(df_yesterday.head(1).index, inplace=True)
    return df_yesterday

def final_Difference(today_Dataframe,df_yesterday):
    today_Dataframe = today_Dataframe
    df_yesterday = df_yesterday
    dfl = pd.DataFrame().reindex_like(today_Dataframe)
    dfl['Expected Supply']  = today_Dataframe['Expected Supply'] - df_yesterday['Expected Supply']
    dfl['Imp Capacity']  = today_Dataframe['Imp Capacity'] - df_yesterday['Imp Capacity']
    dfl['Load + Reserve'] = today_Dataframe['Load + Reserve'] - df_yesterday['Load + Reserve']
    dfl['Surplus']  = today_Dataframe['Surplus'] - df_yesterday['Surplus']
    dfl.reset_index(inplace=True)
    dfl['Effective Date'] = pd.to_datetime(dfl['Effective Date'])
    dfl.drop(dfl.index[0:2], inplace=True)
    return dfl
    

today_Dataframe = saves_Today(stream_data)
df_yesterday = gets_Yesterday()
final_difference = final_Difference(today_Dataframe,df_yesterday)
return_graph(final_difference)  

In [ ]:
## Settings file
from calendar import month
import datetime


def Thirteen_Months():
    csvlineDelta = datetime.datetime.now().date()
    csvlineDelta=csvlineDelta + datetime.timedelta(days=90)
    csvlineDelta = str(csvlineDelta)
    csvlineDelta = csvlineDelta.replace("-", "")
    return csvlineDelta

def sameDay():
    csvlineDelta = datetime.datetime.now().date()
    csvlineDelta=csvlineDelta + datetime.timedelta(days=-1)
    csvlineDelta = str(csvlineDelta)
    csvlineDelta = csvlineDelta.replace("-", "")
    return csvlineDelta

def TwentyFour_Months():
    csvlineDelta = datetime.datetime.now().date()
    csvlineDelta=csvlineDelta + datetime.timedelta(days=730)
    csvlineDelta = str(csvlineDelta)
    csvlineDelta = csvlineDelta.replace("-", "/")
    return csvlineDelta

def todays_date():
    csvline= str(datetime.datetime.now().date())
    DATESTART = csvline
    return DATESTART
    #################

def prevMonth_Date():
    csvlineDelta = datetime.datetime.now().date()
    csvlineDelta=csvlineDelta + datetime.timedelta(days=-30)
    csvlineDelta = str(csvlineDelta)
    return csvlineDelta

def month_Date():
    csvlineDelta = datetime.datetime.now().date()
    csvlineDelta=csvlineDelta + datetime.timedelta(days=30)
    csvlineDelta = str(csvlineDelta)
    return csvlineDelta

def month_DateSLASH(DATE_ONEMONTH):
    DATE_ONEMONTH = DATE_ONEMONTH
    x = DATE_ONEMONTH.replace("-", "/")
    return x

def prevMonth_DateSLASH(DATE_TODAY):
    DATE_TODAY = DATE_TODAY
    x = DATE_TODAY.replace("-", "/")
    return x

def todays_DateSLASH(DATE_TODAY):
    DATE_TODAY = DATE_TODAY
    x = DATE_TODAY.replace("-", "/")
    return x


DATE_TWENTYFOURSLASH = TwentyFour_Months()
DATE_TODAY = todays_date()
DATE_ONEMONTH = month_Date()
DATE_ONEMONTHSLASH = month_DateSLASH(DATE_ONEMONTH)
DATE_ONEMONTHPREV = prevMonth_Date()
DATE_MONTHPREVSLASH = prevMonth_DateSLASH(DATE_ONEMONTHPREV)
DATE_TODAYSLASH = todays_DateSLASH(DATE_TODAY)
DATE_THIRTEEN = Thirteen_Months()
DATE_SAMEDAY=sameDay()
PASSWORD = '**********'
USERNAME = '*******************'


TwentyFourMonthAPI = 278763
enmax_API = 245961
weather_API = 5

#=
#DATESTART7DAYS
DATE_SAMEDAY = int(DATE_SAMEDAY)

# datetime.datetime.now() to get 
# current date as filename.
filename = datetime.datetime.now()
  
# create empty file
def create_file():
    # Function creates an empty file
    # %d - date, %B - month, %Y - Year
    with open(filename.strftime("%d %B %Y")+".txt", "w") as file:
        file.write("")
  
# Driver Code
create_file()